In [ ]:
# read csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Définition du format des logs avec une regex
log_pattern = re.compile(
    r'(?P<date>\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}) '  # Date et heure
    r'SRC=(?P<source_ip>\d+\.\d+\.\d+\.\d+) '          # IP source
    r'DST=(?P<destination_ip>\d+\.\d+\.\d+\.\d+) '     # IP destination
    r'PROTO=(?P<protocol>\w+) '                        # Protocole (TCP/UDP)
    r'DPT=(?P<destination_port>\d+) '                  # Port de destination
    r'ACT=(?P<action>\w+) '                            # Action (Permit/Deny)
    r'RULE=(?P<rule_id>\d+) '                          # ID de la règle firewall
    r'IN=(?P<interface_in>\w+) '                       # Interface d’entrée
    r'OUT=(?P<interface_out>\w+)'                      # Interface de sortie
)

# Charger et parser les logs
log_data = []
with open("log_brute.log", "r", encoding="utf-8") as f:
    for line in f:
        match = log_pattern.search(line)
        if match:
            log_data.append(match.groupdict())  # Ajouter les données extraites

# Convertir en DataFrame
df = pd.DataFrame(log_data)

# Sauvegarder les données nettoyées dans un CSV pour analyse
df.to_csv("parsed_logs.csv", index=False)

# Vérifier les 5 premières lignes
print(df.head())


Analyse et Groupement par IP Source

In [ ]:
# Grouper les logs par adresse IP source
gb = df.groupby("source_ip")

# Créer un nouveau DataFrame avec les statistiques par IP source
ip_stats = pd.DataFrame(index=gb.indices.keys())

# Nombre total de connexions par IP source
ip_stats["total_connexions"] = gb.size()

# Nombre distinct d'IP de destination par IP source
ip_stats["destinations_uniques"] = gb["destination_ip"].nunique()

# Nombre distinct de ports de destination par IP source
ip_stats["ports_distincts"] = gb["destination_port"].nunique()

# Nombre de connexions "Permit" et "Deny"
action_counts = df.groupby(["source_ip", "action"]).size().unstack(fill_value=0)
ip_stats["Permit"] = action_counts["Permit"] if "Permit" in action_counts else 0
ip_stats["Deny"] = action_counts["Deny"] if "Deny" in action_counts else 0

# Nombre de connexions vers des ports inférieurs ou égaux à 1024 (services critiques)
ip_stats["connexions_inf_1024"] = gb.apply(lambda x: (x["destination_port"] <= 1024).sum())

# Nombre de connexions vers des ports administratifs (SSH, FTP, MySQL, RDP)
admin_ports = [21, 22, 3389, 3306]  # FTP, SSH, RDP, MySQL
ip_stats["connexions_admin"] = gb.apply(lambda x: x[x["destination_port"].isin(admin_ports)].shape[0])

# Sauvegarder les statistiques dans un CSV pour analyse
ip_stats.to_csv("ip_stats.csv")

# Afficher les 10 premières lignes du DataFrame
print(ip_stats.head(10))
